In [ ]:
from langchain_community.document_loaders import PyPDFDirectoryLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter, CharacterTextSplitter,TokenTextSplitter
from langchain_community.embeddings.huggingface import HuggingFaceEmbeddings
from langchain_community.retrievers.bm25 import BM25Retriever
from langchain.retrievers import EnsembleRetriever
from langchain_community.vectorstores import Chroma, FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough, RunnableParallel
from sentence_transformers import SentenceTransformer
from tqdm import tqdm
import pandas as pd
import numpy as np
import torch
import os
import gc
import re
from IPython.display import display

DOCS_DIR = './A/A_document'
EMB_MODEL = './bge-large-zh-v1.5'
RERANK_MODEL = "./bge-reranker-large"
PERSIST_DIR = './vectordb' 
QUERY_DIR = './A/A_question.csv'
SUB_DIR = './submit_example.csv'


In [9]:
query = pd.read_csv(QUERY_DIR)
sub = pd.read_csv("./submit_example.csv")
display(query.head(3))
display(sub.head(3))

,ques_id,question
0,1,根据年度报告，2022年中国联通在向数字科技领军企业转变的过程中实现了哪些维度的转型升级？
1,2,告诉我2022年联通产业互联网收入的同比增长速度。
2,3,根据2022年度报告，中国联通的企业定位是什么？


,ques_id,question,answer,embedding
0,1,根据年度报告，2022年中国联通在向数字科技领军企业转变的过程中实现了哪些维度的转型升级？,我们坚定践行网络强国、数字中国、智慧社会战略部署，今天的中国联通，正在从传统运营商加速向数字...,"-0.02707982249557972,-0.009818901307880878,-0...."
1,2,告诉我2022年联通产业互联网收入的同比增长速度。,我们坚定践行网络强国、数字中国、智慧社会战略部署，今天的中国联通，正在从传统运营商加速向数字...,"-0.02707982249557972,-0.009818901307880878,-0...."
2,3,根据2022年度报告，中国联通的企业定位是什么？,我们坚定践行网络强国、数字中国、智慧社会战略部署，今天的中国联通，正在从传统运营商加速向数字...,"-0.02707982249557972,-0.009818901307880878,-0...."


### PDF文档解析和切分

In [ ]:
loader = PyPDFDirectoryLoader(DOCS_DIR)
pages = loader.load_and_split()
pdf_list = os.listdir(DOCS_DIR)

In [6]:
pdf_text = { pdf_page.metadata['source'][-8:]:'' for pdf_page  in pages }
for pdf in tqdm(pdf_list):
    for pdf_page in pages:
        if pdf in pdf_page.metadata['source']:
            pdf_text[pdf] += pdf_page.page_content
        else:
            continue
print('key:pdf value:text')

100%|██████████| 114/114 [00:00<00:00, 6153.33it/s]

key:pdf value:text


In [12]:
def filter_text(text):
    # 页码清除 效果不好
#     page_id_pattern1 = r'\n\d+\s*/\s*\d+\s*\n'
#     page_id_pattern2 = r'\n\d+\n'
#     page_id_pattern3 = r'\n\d+\s*?'

#     page_id_pattern = page_id_pattern1+'|'+page_id_pattern2+'|'+page_id_pattern3
#     text = re.sub(page_id_pattern,'',text)
    
    # '\n', ' ' 删除
    text = text.replace('\n','').replace(' ','')
    
    # 删除页码
    
    # 删除本文档为2024CCFBDC***
    head_pattern = '本文档为2024CCFBDCI比赛用语料的一部分。[^\s]+仅允许在本次比赛中使用。'
    # news_pattern
    pattern1 = r"发布时间：[^\s]+发布人：新闻宣传中心"
    pattern2 = r"发布时间：[^\s]+发布人：新闻发布人"
    pattern3 =  r'发布时间：\d{4}年\d{1,2}月\d{1,2}日'
    news_pattern = head_pattern+'|'+pattern1+'|'+pattern2+'|'+pattern3
    text = re.sub(news_pattern,'',text)
    
    
    # report_pattern
    report_pattern1 = '第一节重要提示[^\s]+本次利润分配方案尚需提交本公司股东大会审议。'
    report_pattern12 = '一重要提示[^\s]+股东大会审议。'
    report_pattern13 = '一、重要提示[^\s]+季度报告未经审计。'
    report_pattern2 = '本公司董事会及全体董事保证本公告内容不存在任何虚假记载、[^\s]+季度财务报表是否经审计□是√否'
    report_pattern3 = '中国联合网络通信股份有限公司（简称“公司”）董事会审计委员会根据相关法律法规、[^\s]+汇报如下：'
    report_pattern = report_pattern1+'|'+report_pattern12+'|'+report_pattern13+'|'+report_pattern2+'|'+report_pattern3
    text = re.sub( report_pattern,'',text)
#     white paper 版本一 效果不好
    # 优先级别 bp1 bp2 bp3
#     bp_pattern_law = '版权声明[^\s]+追究其相关法律责任。'
#     bp_pattern1 = r'目录.*?披露发展报告（\d{4}年）' # 只针对AZ08.pdf
#     bp_pattern2 = r'目录.*?白皮书.*?（\d{4}年）'
#     bp_pattern3 = r'目录.*?白皮书'
#     bp_pattern = bp_pattern_law  +'|'+bp_pattern1+'|'+bp_pattern2+'|'+bp_pattern3
#     text = re.sub(bp_pattern,'',text)
    
#     print(text)
    
    return text
    

In [8]:
for pdf_id in pdf_text.keys():
    pdf_text[pdf_id] = filter_text(pdf_text[pdf_id])
with open('AZ.txt','w',encoding = 'utf-8') as file:
    pdf_all = ''.join(list(pdf_text.values())).encode('utf-8', 'replace').decode('utf-8')
    file.write( pdf_all)    

In [9]:
from langchain_community.document_loaders import TextLoader
loader = TextLoader("AZ.txt",encoding="utf-8")
documents = loader.load()
#分割文本
text_splitter = RecursiveCharacterTextSplitter(        
        chunk_size=245,             
        chunk_overlap=128,
        separators = ["。", "！", "？"],
        keep_separator='end',
    )
docs = text_splitter.split_documents(documents)


In [10]:
len(docs)

4830

### 文本块向量化（比赛限定使用bge-large-zh-v1.5模型）

In [11]:
embeddings = HuggingFaceEmbeddings(model_name=EMB_MODEL, show_progress=True)
vectordb = FAISS.from_documents(   
    documents=docs,
    embedding=embeddings,
)

vectordb.save_local(PERSIST_DIR)

/tmp/ipykernel_18284/1665006100.py:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name=EMB_MODEL, show_progress=True)


Batches:   0%|          | 0/151 [00:00<?, ?it/s]

### 混合检索器

#### bm25 
- k1 较高的 k1 值意味着词频对评分的影响更大。
- b  当 b=1 时，文档长度的影响最大；当b = 0 时，文档长度不影响评分。
- langchain 默认切分英文split()，中文需要jieba分词

In [12]:
import jieba
dense_retriever = vectordb.as_retriever(search_kwargs={"k": 5})
bm25_retriever = BM25Retriever.from_documents(
    docs, 
    k=5, 
    bm25_params={"k1": 1.5, "b": 0.75}, 
    preprocess_func=jieba.lcut
)
ensemble_retriever = EnsembleRetriever(retrievers=[bm25_retriever, dense_retriever], weights=[0.4, 0.6])

Building prefix dict from the default dictionary ...
Dumping model to file cache /tmp/jieba.cache
Loading model cost 0.723 seconds.
Prefix dict has been built successfully.


### 文本召回和重排

In [13]:
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import CrossEncoderReranker
from langchain_community.cross_encoders import HuggingFaceCrossEncoder

def rerank(questions, retriever, top_n=1, cut_len=384):
    rerank_model = HuggingFaceCrossEncoder(model_name=RERANK_MODEL)
    compressor = CrossEncoderReranker(model=rerank_model, top_n=top_n)
    compression_retriever = ContextualCompressionRetriever(
        base_compressor=compressor, base_retriever=retriever
    )
    rerank_answers = []
    for question in tqdm(questions):
        relevant_docs = compression_retriever.invoke(question)
        answer=''
        for rd in relevant_docs:
            answer += rd.page_content
        rerank_answers.append(answer[:245])
    return rerank_answers

questions = list(query['question'].values)
rerank_answers = rerank(questions, ensemble_retriever)
print(rerank_answers[0])

  0%|          | 0/100 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

  1%|          | 1/100 [00:00<00:58,  1.68it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

...

 98%|█████████▊| 98/100 [00:09<00:00, 11.82it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

100%|██████████| 100/100 [00:09<00:00, 10.27it/s]


我们坚定践行网络强国、数字中国、智慧社会战略部署，今天的中国联通，正在从传统运营商加速向数字科技领军企业转变，实现了四个维度的转型升级：一是联接规模和联接结构升维，从过去的连接人为主拓展到连接人机物，大力发展物联网和工业互联网；二是核心功能升维，从以基础连接为主发展到大联接、大计算、大数据、大应用、大安全五大主责主业；三是服务和赋能水平升维，以5G、云计算、大数据、人工智能、区块链为代表的新一代信息技术和实体经济的结合，服务数字政府、数字社会、数字经济的能力不断增强；四是发展理念升维，


### 提交

### Embedding

In [14]:
def emb(answers, emb_batch_size = 4):
    model = SentenceTransformer(EMB_MODEL, trust_remote_code=True)
    all_sentence_embeddings = []
    for i in tqdm(range(0, len(answers), emb_batch_size), desc="embedding sentences"):
        batch_sentences = answers[i:i+emb_batch_size]
        sentence_embeddings = model.encode(batch_sentences, normalize_embeddings=True)
        all_sentence_embeddings.append(sentence_embeddings)
    all_sentence_embeddings = np.concatenate(all_sentence_embeddings, axis=0)
    print('emb_model max_seq_length: ', model.max_seq_length)
    print('emb_model embeddings_shape: ', all_sentence_embeddings.shape[-1])
    del model
    gc.collect()
    torch.cuda.empty_cache()
    return all_sentence_embeddings

all_sentence_embeddings = emb(rerank_answers)


embedding sentences: 100%|██████████| 25/25 [00:00<00:00, 39.90it/s]


emb_model max_seq_length:  512
emb_model embeddings_shape:  1024


In [15]:
sub['answer'] = rerank_answers
sub['embedding']= [','.join([str(a) for a in all_sentence_embeddings[i]]) for i in range(len(all_sentence_embeddings))]
sub.to_csv('submit.csv', index=None)
sub.head()

,ques_id,question,answer,embedding
0,1,根据年度报告，2022年中国联通在向数字科技领军企业转变的过程中实现了哪些维度的转型升级？,我们坚定践行网络强国、数字中国、智慧社会战略部署，今天的中国联通，正在从传统运营商加速向数字...,"-0.029152686,-0.010147623,-0.0077438904,0.0005..."
1,2,告诉我2022年联通产业互联网收入的同比增长速度。,（三）拥抱创新，释放全新增长空间在确保稳定发展的基础之上，我们持续从深层次、多领域推进创新转...,"-0.035577893,-0.009453767,-0.008823524,0.02334..."
2,3,根据2022年度报告，中国联通的企业定位是什么？,中国联合网络通信股份有限公司2022年年度报告1公司代码：600050公司简称：中国联通中国...,"-0.037686788,-0.024368605,-0.06751171,0.001446..."
3,4,2022年联通在“大联接”和“大数据”业务上取得了什么成果？,大联接价值生长，物联网连接数达到3.9亿，5G连接新增市场份额接近七成，在行业内率先实现了“...,"-0.015162307,-0.0038760568,-0.023473836,0.0059..."
4,5,2022年上半年，联通在精品网络建设上有什么成果？,"宽带接入用户净增440万户，总数达到9,944万户，净增用户数创近10年同期新高，融合渗透率...","-0.018238459,-0.024857994,-0.05196209,0.008531..."
